# PlayHQ Season Competition: Team & Fixture Extraction

This notebook can be used to:

1. Find the seasons available under an orgaization (e.g., Coburg Association).
2. Find season ID from the season name, and vice-versa.
3. Extract all teams in a season competition (and save them into a CSV file).
4. Extract the fixtures of every team in the competition so far. If the games have been played, the result will be there too.

All the data is from [PlayHQ](http://playhq.com/) via its Public [API](https://support.playhq.com/hc/en-au/sections/4405422358297-PlayHQ-APIs). Detailed reference documentation for PlayHQ API can be found [here](https://docs.playhq.com/tech).

This notebook was first used to support the scheduler season capstone project in 2023.

**Contact:** Sebastian Sardina (sssardina@gmail.com)

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import json
import re
import os
import csv

import playhq as phq

## 1. Configuration and set-up

We first configure and set-up the application. This means reading configuration variables from a config file and setting the game day.

So, first of all, specify the following information:

1. Configuration file for the club and season.
2. Game dates interval to scrape.

In [ ]:
# Change this to import your club's own configuration
# from config_bmc_s22 import *
from config_bmc_s23 import *
# from config_cba import *

# Create phq_club object
phq_club = phq.PlayHQ(CLUB_NAME, ORG_ID, X_API_KEY, X_TENANT, TIMEZONE, tapp_team_name, tapp_game_name)

print(f"Club name: {CLUB_NAME} (org. id: {ORG_ID})")
print("X-tenant:", X_TENANT, "x-api-key:", X_API_KEY)

print("Timezone:", TIMEZONE)
print("PlayHQ Club fixture:", PLAYHQ_SEASON_URL)


OUTPUT_ROOT_PATH = "fixtures-out"
if not os.path.exists(OUTPUT_ROOT_PATH):
    print("ERROR: Output path does not exist:", OUTPUT_ROOT_PATH)


## 2. Get season id from season name

Find the season id from the season name, if we don't know it. It will also print the JSON received so one can manually check the seasons available for the organization.

In [ ]:
# get competition id
SEASON = "Winter 2023"
SEASON = "Summer 2023/24"
# SEASON = None   # use this if you don't know and just want to list them all

season_id = None
competition_id = None
for data_json in phq_club.get_json(f"organisations/{ORG_ID}/seasons"):
    print(json.dumps(data_json, sort_keys=True, indent=4))

    for x in data_json['data']:
        if SEASON is None:
            continue
        if x['name'] == SEASON:
            season_id = x['id']
            # competition_id = x['competition']['id']
            print(f'Seasons *{SEASON}* found with id: {season_id}')



## 3. Set the season id to scrape

First set the season id we want to scrape. We can get it from the name using the above cell, but it has to be set manually here. 

Once set, find the season name and set the output path where all output files (including the fixture per team) will be saved.

In [ ]:
season_id = "a94981b4-75b7-429f-9005-915182ab6153" # Victorian Junior Domestic - Summer 2022/23
season_id = "cdbe3065-2a32-4c6d-8771-f8fae3fa7611" # Victorian Junior Domestic - Winter 2023
season_id = "2485b3f7-55de-4b88-a3b6-527c36717fee" # Victorian Junior Domestic - Summer 2023/24

# season_id = "78824ad1-0ca6-46c5-9440-dbe20c948b2f"  # VJBL 2023
# season_id = "8d724c0a-3a0f-4da7-92f8-2e2053a6707b"  # National Junior 2023

season_name = None
for data_json in phq_club.get_json(f"organisations/{ORG_ID}/seasons"):
    # print(json.dumps(data_json, sort_keys=True, indent=4))

    for x in data_json['data']:
        if x['id'] == season_id:
            season_name = x['name']
            break


print(f'Found season name for *{season_id}*: {season_name}')

OUTPUT_PATH = os.path.join(OUTPUT_ROOT_PATH, season_name.lower().replace(' ', '_').replace('/', '-'))

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

print(f'Output PATH to store fixtures ready: {OUTPUT_PATH}')


## 4. Extract participating teams

Get all the teams participating in the competition. Save the list to file `teams.csv`

In [ ]:
teams = []
for data_json in phq_club.get_json(f"seasons/{season_id}/teams"):
    print(data_json)
    if data_json['data'][0]['club'] is None:
        break
    for t in data_json['data']:
        # print(t)
        team = {}

        team['id'] = t['id']
        team['club'] = t['club']['name'] if t['club'] is not None else "Unknown"
        if t['grade'] is None:
            continue    # skip teams without grade
        team['grade'] = t['grade']['name']
        team['name'] = t['name']

        teams.append(team)
    # print(json.dumps(data_json, sort_keys=True, indent=4))

print("Number of teams extracted:", len(teams))
print(teams[10])


fieldnames = ["id", "club", "grade", "name"]
with open(os.path.join(OUTPUT_PATH, "teams.csv"), 'w', encoding='UTF8', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(teams)


## 5. Extract fixture for each team

Now extract the fixture so far of every team in the competition. Store each team fixture in a JSON file. Note that games will appear in two files always: for the home and away teams.

In [ ]:
# team_id = "139ecf7c-55e5-411f-bbce-4bceb321f342"
# team_id="b55cb46c-8415-4b6e-9ac5-ad38b02e5b35"

for team in teams:
    team_id = team['id']
    standard_name = re.sub(r'[\W_]', "_", team['name'])
    file_name = os.path.join(OUTPUT_PATH, standard_name + ".json") 

    if os.path.exists(file_name):
        continue
    print("Processing team:", team['name'])

    games = {}
    games["data"] = []

    for data_json in phq_club.get_json(f"teams/{team_id}/fixture"):
        # if data_json['data'][0]['club'] is None:
        #     break
        games["data"] = games["data"] + data_json['data']

    with open(file_name, "w") as f:
        f.write(json.dumps(games, sort_keys=True, indent=4))


print("Number of teams fixture extracted:", len(teams))
# print(teams[10])

To print the fixture of a particular team:

In [ ]:
team_id = '10b5d298-2de3-4296-a360-70c8d729fbc5'

for data_json in phq_club.get_json(f"teams/{team_id}/fixture"):
    print(data_json)